In [1]:
from init import *

In [2]:
a = {
    "account": test_account1,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":test_account1,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [8]:
test_account1, config.main_token

('wkpmdjdsztyu', 'TNT')

In [44]:
code = r'''
#include <vector>
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/asset.hpp>
#include <eosio/transaction.hpp>

using namespace std;
using namespace eosio;

#define N(name) name##_n.value

struct transfer {
    name from;
    name to;
    asset amount;
    string memo;
    EOSLIB_SERIALIZE( transfer, (from)(to)(amount)(memo) )
};


extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if (N("sayhello") == action) {
            name sender("wkpmdjdsztyu");
            struct action a;
            a.account = "eosio.token"_n;
            a.name = "transfer"_n;
            a.authorization.push_back({sender, "active"_n});
            
            transfer t;
            t.from = sender;
            t.to = "eosio"_n;
            t.amount.amount = 1000; //0.1 TNT
            t.amount.symbol = symbol("TNT", 4);
            a.data = eosio::pack<transfer>(t);
            
            transaction trx;
            trx.actions.push_back(a);
            trx.delay_sec = 3; //delay 2 seconds
            trx.send(1, sender);
        }
    }
}
'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [45]:
uuosapi.get_balance(test_account1),uuosapi.get_balance('eosio')

(54.3551, 798712559.6884)

In [46]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])


1844


In [47]:
uuosapi.get_balance(test_account1),uuosapi.get_balance('eosio')

(54.2551, 798712559.7884)